## Fungsi Linear

Fungsi merupakan cara untuk memetakan suatu nilai atau objek tertentu ke nilai yang lain. Notasi fungsi $f: \mathbb{R}^m \rightarrow \mathbb{R}$ berarti pemetaan nilai dari vektor berdimensi $m$ menjadi suatu nilai skalar.

Sebagai contoh, misalkan terdapat vektor $\mathbf{x} = (x_1, \ldots, x_4)$, berikut salah satu bentuk fungsi yang mungkin untuk memetakan ke nilai skalar:

$$
y = f(\mathbf{x}) = x_1 + x_2 - x^2_4
$$

Python menyediakan beberapa cara untuk mendefinisikan fungsi.

**Lambda function**: fungsi yang didefinisikan tanpa nama.

In [1]:
import jax.numpy as jnp

In [2]:
f = lambda x: x[0] + x[1] - x[3] ** 2
f(jnp.array([-1.0, 0.0, 1.0, 2.0]))

Array(-5., dtype=float32)

**def function**: pendefinisian fungsi untuk kebutuhan yang lebih kompleks, dapat mengakomodasi multi-input dan multi-output.

In [3]:
def simple_function(x):
    """
    Args:
        x (ndarray): a vector of length 4
    
    Returns:
        y (float): a scalar value
    """
    y = x[0] + x[1] - x[3] ** 2
    return y

simple_function(jnp.array([-1.0, 0.0, 1.0, 2.0]))

Array(-5., dtype=float32)

### Superposisi

Superposisi merupakan sebuah sifat yang dapat memvalidasi apakah suatu fungsi merupakan fungsi linear. Misal terdapat vektor $\mathbf{x}, \mathbf{y} \in \mathbb{R}^m$, skalar $\alpha, \beta \in \mathbb{R}$, dan fungsi $f: \mathbb{R}^m \rightarrow \mathbb{R}$, persamaan superposisi dapat ditulis sbb:

$$
f(\alpha \mathbf{x} + \beta \mathbf{y}) = \alpha f(\mathbf{x}) + \beta f(\mathbf{y}).
$$

*Penting*: fungsi $f: \mathbb{R}^m \rightarrow \mathbb{R}$ merupakan fungsi linear jika memenuhi sifat pada persamaan di atas!

Contoh: cek linearitas fungsi inner product $f(\mathbf{x}) = \mathbf{a}^\top \mathbf{x} $.

In [7]:
import jax
f = lambda a, x: jnp.dot(a, x)

a = jnp.array([1.0, 2.0, 3.0, 4.0, 5.0])
key = jax.random.PRNGKey(0)
key_x, key_y = jax.random.split(key)
x = jax.random.uniform(key_x, (5,))
y = jax.random.uniform(key_y, (5,))

alpha = 3.0
beta = 5.0

print(f"a: {a}")
print(f"x: {x}")

lhs = f(a, alpha * x + beta * y)
print(f"lhs: {lhs}")

rhs = alpha * f(a, x) + beta * f(a, y)
print(f"rhs: {rhs}")

print(jnp.allclose(lhs, rhs))

a: [1. 2. 3. 4. 5.]
x: [0.8423141  0.18237865 0.2271781  0.12072563 0.19181347]
lhs: 31.77120590209961
rhs: 31.77120590209961
True


### Aproksimasi Taylor

Dalam banyak kasus, fungsi skalar $f: \mathbb{R}^m \rightarrow \mathbb{R}$ dapat dan perlu diaproksimasi dengan fungsi linear atau *affine*, dengan syarat fungsi $f$ dapat diturunkan (*differentiable*). Aproksimasi Taylor menggunakan turunan untuk membentuk suatu fungsi linear $g(\mathbf{x})$ yang mendekati fungsi $f(\mathbf{x})$ (yang tidak harus bersifat linear).

(First-order) aproksimasi Taylor dari fungsi $f$ pada titik ($\mathbf{z} \in \mathbb{R}^m$) atau yang dekat dengan itu didefinisikan sbb:

$$
f^\prime(\mathbf{x}) = f(\mathbf{z}) + \frac{\partial f}{\partial x_1} (z_1) + \frac{\partial f}{\partial x_2} (z_2) + \cdots + \frac{\partial f}{\partial x_m} (z_m)
$$

Dalam format vektor dapat dituliskan menjadi:
$$
f^\prime(\mathbf{x}) = f(\mathbf{z}) + \nabla_x f(\mathbf{z})^\top (\mathbf{x} - \mathbf{z})
$$

dimana 
$
\nabla_x f(\mathbf{z})= 
(
\frac{\partial f}{\partial x_1 (z_1)},
\frac{\partial f}{\partial x_1 (z_2)},
\cdots,
 \frac{\partial f}{\partial x_m (z_m)}
)
$ merupakan vektor gradien.

**Contoh**. Terdapat sebuah fungsi non-linear $f: \mathbb{R}^2 \rightarrow \mathbb{R}$,
$$f(\mathbf{x}) = x_1 + \exp(x_2 - x_1)$$

Fungsi linear hasil aproksimasi Taylor (first-order) pada titik atau yang dekat dengan $\mathbf{z} = (1, 2)$ adalah sbb:
$$
f^\prime(\mathbf{x}) = f((1, 2)) + \nabla_x f((1, 2))^\top (\mathbf{x} - (1, 2)) = 3.7183 + (-1.7183, 2.7183)^\top (\mathbf{x} - (1,2))
$$.

Dapat dicek bahwa:
- $\mathbf{x} = (1,2) \rightarrow$  $f(\mathbf{x}) = 3.7183$ dan $f^\prime(\mathbf{x}) = 3.7183$ 
- $\mathbf{x} = (0.96,1.98) \rightarrow$  $f(\mathbf{x}) = 3.7332$ dan $f^\prime(\mathbf{x}) = 3.7326$ 
- $\mathbf{x} = (1.10,2.11) \rightarrow$  $f(\mathbf{x}) = 3.8456$ dan $f^\prime(\mathbf{x}) = 3.8455$ 

In [8]:
# Define f function
f = lambda x: x[0] + jnp.exp(x[1] - x[0])

# Define the first-order gradient of f (using JAX autodiff as well for demonstration)
grad_f_jax = jax.grad(f)

# Manual gradient as per example
grad_f_manual = lambda z: jnp.array([1 - jnp.exp(z[1] - z[0]), jnp.exp(z[1] - z[0])])

# Taylor approximation
z = jnp.array([1.0, 2.0])
f_prime = lambda x: f(z) + jnp.dot(grad_f_manual(z), (x - z))

In [6]:
x = jnp.array([1.0, 2.0])
print(f"{x} -> {f(x)} and {f_prime(x)}")

x = jnp.array([0.96, 1.98])
print(f"{x} -> {f(x)} and {f_prime(x)}")

x = jnp.array([1.10, 2.11])
print(f"{x} -> {f(x)} and {f_prime(x)}")

[1. 2.] -> 3.7182817459106445 and 3.7182817459106445
[0.96 1.98] -> 3.7331948280334473 and 3.73264741897583
[1.1  2.11] -> 3.8456006050109863 and 3.8454642295837402
